<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/DUDE-Z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
import timeit
import os
import timeit
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
condacolab.install()
!mamba install openbabel

time: 4.53 s (started: 2023-10-04 16:19:38 +00:00)


In [ ]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j2

time: 3min 14s (started: 2023-10-04 16:20:22 +00:00)


In [ ]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!
time: 8.99 s (started: 2023-10-04 16:23:36 +00:00)


In [ ]:
folder = "/content/gdrive/MyDrive/Projects_Data/LiBELa/DUDE-Z" # @param {type:"string"}

drive.mount('/content/gdrive')
%cd $folder
%cd scripts
tlist = open('targets.dat', 'r')
targets=[]
for line in tlist:
  targets.append(line)
tlist.close()


Mounted at /gdrive
/gdrive/MyDrive/Projects_Data/LiBELa/DUDE-Z
/gdrive/MyDrive/Projects_Data/LiBELa/DUDE-Z/scripts
time: 34 s (started: 2023-10-04 16:19:46 +00:00)


In [ ]:
#@title Docking parameters {display-mode: "form"}

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = False;
Input.use_grids = True
Input.grid_prefix='McGrid'
Input.load_grid_from_file = True;
Input.write_mol2 = False
Input.atom_limit = 100 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.4 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-5;
Input.min_delta = 1E-3;
Input.dock_min_tol = 1E-5;
search_box = 20.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 20 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

Input.use_overlay_cutoff = True # @param {type:"boolean"}
Input.overlay_cutoff = 0.6 #@param {type:"number"}
Input.use_smiles = True;

time: 6.99 ms (started: 2023-10-04 16:23:45 +00:00)


In [ ]:

#@title Reading molecular files and Docking {display-mode: "form"}

targets=['GLCM']

for target in (targets):
  %cd $folder/targets/$target
  !mkdir -p SF0_mma
  %cd SF0_mma
  base_prefix=target;

# get the list of multimol files
  multimol_list=[]
  for item in os.listdir('../'):
    if item.startswith('multimol.'):
      multimol_list.append('../' + item)

  print('There are %d chuncks of ligands to dock.' % len(multimol_list))


  REC = Mol2(Input, '../rec.mol2.gz')
  RefLig = Mol2(Input, '../lig.mol2.gz')

  Writer = WRITER(Input)
  Coord = COORD_MC()
  HB = FindHB()
  Ene  = Energy2(Input)

  for i in range(len(REC.residue_pointer)-1):
    HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
  HB.find_ligandHB('../lig.mol2.gz', RefLig);
  print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));


  center = Coord.compute_com(RefLig)

  print()
  start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
  print('Starting energy: %7.3f kcal/mol' % start_energy);
  print()
  print('Generating grids. This may take a while..')

  Grids = Grid(Input, Writer)
  Grids.load_grids_from_file();
  print('Grids loaded.')
  grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
  print('Grid original energy: %7.3f kcal/mol' % grid_energy);
  print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));

  count=0;
  count_chunck=0;
  for multimol in (multimol_list):
    print("Processing chunck %s ..." % multimol);
    count_chunck += 1;
    Input.output=(base_prefix+'_'+str(count_chunck))
    molecules=[]
    smiles_list = open(multimol, 'r')
    for line in smiles_list:
      line2 = line.split()
      molecules.append(line2[0])
    smiles_list.close()
    for mol in molecules:
      Lig2 = Mol2();
      if Lig2.parse_smiles(Input, mol, 'Lig'):
        count += 1;
        Dock.run(REC, Lig2, RefLig, center, Input, Grids, count);
      else:
        print('Molecule %s skipped' % molecules[mol]);


/gdrive/MyDrive/Projects_Data/LiBELa/DUDE-Z/targets/MT1
/gdrive/MyDrive/Projects_Data/LiBELa/DUDE-Z/targets/MT1/SF0_mma
There are 4 chuncks of ligands to dock.
The receptor has    14 /    14 HB donors/acceptors around the active site.

Starting energy: -36.230 kcal/mol

Generating grids. This may take a while..
Grids loaded.
Grid original energy: -35.833 kcal/mol
Grid error:   1.095 %
Processing chunck ../multimol.00 ...
Processing chunck ../multimol.01 ...
Processing chunck ../multimol.02 ...
Processing chunck ../multimol.03 ...
time: 3h 18min 59s (started: 2023-10-04 16:24:05 +00:00)


In [ ]:
#@title Computing Enrichment {display-mode: "form"}
! more McLiBELa.log | grep LIG | sort -g -k 9 > sorted_list
sorted_file=open('sorted_list', 'r')
nlig = ! wc -l ../ligands.smi | awk '{print $1}'
nlig=int(nlig)
ndec = ! wc -l ../decoys.smi | awk '{print $1}'
ndec=int(ndec)
found_decoys=0
found_ligands=0
x=[]
y=[]
for line in sorted_file:
  line2 = line.split()
  if (line2[0] <= nlig):
    found_ligands += 1;
  else:
    found_decoys += 1;
  x.append(found_decoys/ndec);
  y.append(found_ligands/lig);

sorted_file.close()

plt.plot(x, y, 'o', ls='-', ms=4))
plt.ylabel('Fraction of Actives')
plt.xlabel('Fraction of Decoys')
plt.title(base_prefix)
plt.show()

